In [1]:
!pip install -q transformers

     |████████████████████████████████| 4.0 MB 9.2 MB/s 
     |████████████████████████████████| 6.6 MB 58.3 MB/s 
     |████████████████████████████████| 895 kB 59.6 MB/s 
     |████████████████████████████████| 596 kB 52.6 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 


In [ ]:
"""
import gc
torch.cuda.empty_cache()
gc.collect()
torch.cuda.memory_summary(device=None, abbreviated=False)
"""

NameError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/544final/')

In [4]:
import pandas as pd

In [ ]:
"""
filename = "./ratings.csv"
df = pd.read_csv(filename, encoding='utf-8')
df.drop(columns=['userId','productId','timestamp','title'], inplace=True)

df.dropna(inplace=True)
import sklearn
from sklearn.model_selection  import train_test_split

df2 = df[df['rating']==1].sample(n=16000, replace=False)
for i in range(2,6):
  df2 = pd.concat([df2, df[df['rating']==i].sample(n=16000, replace=False)])
"""

'\nfilename = "./ratings.csv"\ndf = pd.read_csv(filename, encoding=\'utf-8\')\ndf.drop(columns=[\'userId\',\'productId\',\'timestamp\',\'title\'], inplace=True)\n\ndf.dropna(inplace=True)\nimport sklearn\nfrom sklearn.model_selection  import train_test_split\n\ndf2 = df[df[\'rating\']==1].sample(n=16000, replace=False)\nfor i in range(2,6):\n  df2 = pd.concat([df2, df[df[\'rating\']==i].sample(n=16000, replace=False)])\n'

In [ ]:
"""
del df
# train, val, test: 8-1-1
train_df, test_df = train_test_split(df2, test_size=0.2)
test_df, val_df = train_test_split(test_df, test_size=0.5)

train_df.to_csv("./train_df_large.csv",index=None)
val_df.to_csv("./val_df_large.csv",index=None)
test_df.to_csv("./test_df_large.csv",index=None)
"""

'\ndel df\n# train, val, test: 8-1-1\ntrain_df, test_df = train_test_split(df2, test_size=0.2)\ntest_df, val_df = train_test_split(test_df, test_size=0.5)\n\ntrain_df.to_csv("./train_df_large.csv",index=None)\nval_df.to_csv("./val_df_large.csv",index=None)\ntest_df.to_csv("./test_df_large.csv",index=None)\n'

In [5]:
"""
train_df = pd.read_csv("./train_df.csv")
val_df = pd.read_csv("./val_df.csv")
test_df = pd.read_csv("./test_df.csv")
"""
train_df = pd.read_csv("./train_df_large.csv")
val_df = pd.read_csv("./val_df_large.csv")
test_df = pd.read_csv("./test_df_large.csv")

In [ ]:
test_df.shape

(1600, 6)

In [6]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence


class RatingDataSet(Dataset):
  def __init__(self, train_df, val_df, test_df, maxLength=256, batch_size=16):
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    self.tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
    #self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    self.max_len = maxLength
    self.batch_size = batch_size
    self.train_data = None
    self.val_data = None
    self.test_data = None
    self.init_data()

  def init_data(self):
    self.train_data = self.load_data(self.train_df)
    self.val_data = self.load_data(self.val_df)
    self.test_data = self.load_tst_data(self.test_df)

  def load_data(self, df):
    feature_ids = []
    token_ids = []
    mask_ids = []
    labels = []
    
    titles = df['title'].to_list()
    comments = df['comment'].to_list()
    ratings = df['rating'].to_list()
    for (title, comment, rating) in zip(titles, comments, ratings):
      #ref: https://huggingface.co/docs/transformers/main/en/model_doc/bert#transformers.BertForMaskedLM
      feature = self.tokenizer.encode_plus(title+comment,
                        add_special_tokens=True, 
                        max_length=self.max_len,
                        truncation=True, 
                        padding='max_length', 
                        return_attention_mask=True,
                        return_tensors='pt')
      #,return_tensors='pt'
      feature_ids.append(torch.squeeze(feature['input_ids']))
      token_ids.append(torch.squeeze(feature['token_type_ids']))
      mask_ids.append(torch.squeeze(feature['attention_mask']))

      labels.append(rating-1)

    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    feature_ids = pad_sequence(feature_ids, batch_first=True)

    labels = torch.tensor(labels, dtype=int)
    dataset = TensorDataset(feature_ids, mask_ids, token_ids, labels)
    return dataset

  def load_tst_data(self, df):
    feature_ids = []
    token_ids = []
    mask_ids = []
    titles = df['title'].to_list()
    comments = df['comment'].to_list()
    for (title, comment) in zip(titles,comments):
      feature = self.tokenizer.encode_plus(title+comment,
                        add_special_tokens=True, 
                        max_length=self.max_len,
                        truncation=True, 
                        padding='max_length', 
                        return_attention_mask=True,
                        return_tensors='pt')
      feature_ids.append(torch.squeeze(feature['input_ids']))
      token_ids.append(torch.squeeze(feature['token_type_ids']))
      mask_ids.append(torch.squeeze(feature['attention_mask']))
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    feature_ids = pad_sequence(feature_ids, batch_first=True)
    dataset = TensorDataset(feature_ids, mask_ids, token_ids)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=self.batch_size
    )
    print(len(train_loader.dataset))

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=self.batch_size
    )

    test_loader = DataLoader(
        self.test_data,
        shuffle = False,
        batch_size = self.batch_size
    )
    print(len(test_loader.dataset))
    return train_loader, val_loader, test_loader
  
Ratingds = RatingDataSet(train_df, val_df, test_df)
del train_df
del val_df
del test_df

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

1600


In [7]:
lr = 1e-5
epochs = 5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=5, hidden_dropout_prob=0.3, attention_probs_dropout_prob=0.3)
#, problem_type="single_label_classification"
#model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased",num_labels=5)
model.to(device)
path='./model_revised.pt'
if os.path.exists(path):
  checkpoint = torch.load(path)
  model.load_state_dict(checkpoint)
  print("model load")

cuda:0


Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

model load


In [ ]:
from transformers import AdamW
import torch
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=lr, correct_bias=False)
path = './model_revised512_2.pt'


def multi_acc(y_pred, y_test):
  #acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test.argmax(dim=1)).sum().float() / float(y_test.size(0))
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc
from torch.nn import BCEWithLogitsLoss, BCELoss
def train(model, train_loader, val_loader, optimizer):  
  total_step = len(train_loader)
  global_dev_loss = float('inf')
  for epoch in range(epochs):
    model.train()
    total_train_loss = 0
    total_train_acc  = 0

    for batch_idx, (feature_ids, mask_ids, token_ids, label) in enumerate(train_loader):
      optimizer.zero_grad()
      feature_ids = feature_ids.to(device)
      mask_ids = mask_ids.to(device)
      token_ids = token_ids.to(device)
      
      #label= torch.unsqueeze(label,1)
      #label = label.long()
      label = label.to(device)
      #label = torch.zeros(8,5,device=device).scatter_(1,torch.cuda.LongTensor(label),1)
      
      loss, prediction = model(feature_ids, token_type_ids=None, attention_mask=mask_ids, labels=label).values()
      
      acc = multi_acc(prediction, label)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, (feature_ids, mask_ids, token_ids, label) in enumerate(val_loader):
        optimizer.zero_grad()
        feature_ids = feature_ids.to(device)
        mask_ids = mask_ids.to(device)
        token_ids = token_ids.to(device)
        #label= torch.unsqueeze(label,1)
        #label = label.long()
        label = label.to(device)
        #label = torch.zeros(8,5,device=device).scatter_(1,torch.cuda.LongTensor(label),1)
        loss, prediction = model(feature_ids, token_type_ids=None, attention_mask=mask_ids, labels=label).values()
        acc = multi_acc(prediction, label)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    if val_loss<global_dev_loss:
      torch.save(model.state_dict(), path)

    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [8]:
train_loader,val_loader, test_loader = Ratingds.get_data_loaders()
del Ratingds

12798
1600


In [ ]:
"""
import gc
torch.cuda.empty_cache()
gc.collect()
torch.cuda.memory_summary(device=None, abbreviated=False)
"""

train(model, train_loader, val_loader, optimizer)

Epoch 1: train_loss: 1.1670 train_acc: 0.4701 | val_loss: 1.0344 val_acc: 0.5613
Epoch 2: train_loss: 1.0051 train_acc: 0.5537 | val_loss: 1.0229 val_acc: 0.5644
Epoch 3: train_loss: 0.9567 train_acc: 0.5709 | val_loss: 1.0316 val_acc: 0.5575
Epoch 4: train_loss: 0.9336 train_acc: 0.5859 | val_loss: 1.0491 val_acc: 0.5463


KeyboardInterrupt: ignored

In [9]:
def prediction(model, device, test_loader):
  model.eval()
  results = []
  with torch.no_grad():
    for batch_idx, (feature_ids, mask_ids, token_ids) in enumerate(test_loader):
        optimizer.zero_grad()
        feature_ids = feature_ids.to(device)
        mask_ids = mask_ids.to(device)
        token_ids = token_ids.to(device)

        pred = model(feature_ids, token_type_ids=None, attention_mask=mask_ids).values()
        pred = torch.log_softmax(list(pred)[0], dim=1).argmax(dim=1)
        results += pred.tolist()
  return results


from transformers import AdamW
import torch
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=lr, correct_bias=False)
path='./model_revised.pt'
if os.path.exists(path):
  checkpoint = torch.load(path)
  model.load_state_dict(checkpoint)
  print("model load")
#optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)
results = prediction(model, device, test_loader)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


model load


In [10]:
test_df = pd.read_csv("./test_df_large.csv")

In [11]:
a = test_df['rating'].tolist()
c = 0
for i in range(len(a)):
  if int(a[i])==results[i]+1:
    c+=1
print(c/len(a))

0.549375


In [14]:
import numpy as np
labels = np.array(test_df['rating'].tolist())
predictions = np.array(results)+1
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(labels, predictions, squared=False)
print(rmse)

0.8774964387392122
